# Example of FIRe scoring system on 10 menus

In [33]:
import pandas as pd
import simplejson as json
from analysis.simple_spell import *
from scrapers.TestCorpusScraper_editing import *
from analysis.scoring import *
from analysis.Indexes import *
import warnings
warnings.filterwarnings('ignore')


## Background corpus 

The following code reads the corpus we created to assess authencity of restaurants' menu. We display some of the main features of this corpus as follows.

In [4]:
# Importing the background corpus
with open('data/Background Corpora.json', 'r') as j:
        Background_Corpora_Json = json.load(j)


In [78]:
all_tokens = get_alltokens(Background_Corpora_Json)
print(all_tokens[:30])

['spaghetti', 'guanciale', 'tuorli', 'uova', 'medie', 'pecorino', 'romano', 'pepe', 'nero', 'preparare', 'carbonara', 'cominciate', 'mettendo', 'fuoco', 'pentola', 'lacqua', 'salata', 'cuocere', 'pasta', 'frattempo', 'eliminate', 'cotenna', 'tagliatelo', 'prima', 'fette', 'poi', 'striscioline', 'spesse', 'circa', 'cm']


In [79]:
all_ngrams = ngrams_it(Background_Corpora_Json)
print(all_ngrams[:20])

[['spaghetti', 'guanciale', 'tuorli'], ['guanciale', 'tuorli', 'di'], ['tuorli', 'di', 'uova'], ['di', 'uova', 'medie'], ['uova', 'medie', 'pecorino'], ['medie', 'pecorino', 'romano'], ['pecorino', 'romano', 'pepe'], ['romano', 'pepe', 'nero'], ['pepe', 'nero', 'per'], ['nero', 'per', 'preparare'], ['per', 'preparare', 'gli'], ['preparare', 'gli', 'spaghetti'], ['gli', 'spaghetti', 'alla'], ['spaghetti', 'alla', 'carbonara'], ['alla', 'carbonara', 'cominciate'], ['carbonara', 'cominciate', 'mettendo'], ['cominciate', 'mettendo', 'sul'], ['mettendo', 'sul', 'fuoco'], ['sul', 'fuoco', 'una'], ['fuoco', 'una', 'pentola']]


In [90]:
fr_grams = freq_grams(all_ngrams)
fr_grams

TypeError: unhashable type: 'list'

In [99]:
words_freq = tf(Background_Corpora_Json)
words_freq[:20]

['spaghetti', 'guanciale', 'tuorli', 'uova', 'medie']
[41, 25, 22, 71, 16]


In [11]:
doc_terms = tf_doc(Background_Corpora_Json)
doc_terms[:5]

TypeError: unhashable type: 'slice'

In [12]:
term_docs = inv_index(Background_Corpora_Json, all_tokens)
term_docs[:5]

KeyboardInterrupt: 

In [ ]:
grams_dic = term_grams(all_tokens, all_ngrams)
grams_dic[:5]

## Examples from our Test corpus

Here we read and analyze the 10 menus from our scraped corpus and provide some insights as follows

In [13]:
# Importing the background corpus
with open('data/Test Corpus.json', 'r') as j:
        Test_Corpus_Json = json.load(j)


In [14]:
all_tokens_test = get_tokens_test(Test_Corpus_Json)
all_tokens_test[:20]

TypeError: unhashable type: 'slice'

In [15]:
menus = set_menu(all_tokens_test)
menus[:5]

TypeError: unhashable type: 'slice'

### Check for typos in the menus

In [143]:
final_results = {}
for key, value in menus.items():
    result, errs = checker(words_freq.keys(), value)
    final_results[key] = result, errs

In [87]:
def scoring(result):
    if result <= 5:
        scores = 5
        img = mpimg.imread('images/nonna-open.jpg')
    elif result >= 6 and result <= 10:
        scores = 4
        img = mpimg.imread('images/Red-Sauce-Raos.jpg')
    elif result >= 11 and result <= 15:
        scores = 3
        img = mpimg.imread('images/rosie-s-italian-grille.jpg')
    elif result >= 16 and result <= 20:
        scores = 2
        img = mpimg.imread('images/olive_garden.jpg')
    else:
        scores = 1
        img = mpimg.imread('images/pizza_deep.jpg')

    return scores, img

### Assign our scores to each menu

In [163]:
dic_scoring = {}
for key, value in final_results.items():
    score, img = scoring(value[0])
    dic_scoring[key] = value, score, img

In [164]:
df_errors = []
for key, value in dic_scoring.items():
    df = {'Restaurant': key, 'Errors': value[0][1], '# Errors': value[0][0], 'Score': value[1]}
    df_errors.append(df)
df_errors = pd.DataFrame(df_errors, columns=['Restaurant', 'Errors','# Errors', 'Score'])

### Here's how these restaurants performed and what mistakes our program detected

In [165]:
df_errors.sort_values('Score', ascending=False)

,Restaurant,Errors,# Errors,Score
5,"Il Canale - Washington, DC","[georgetown, rolle, tunnarella, ciliegina, imb...",5,5
1,"La Storia - Chicago, IL","[neri, orecchitte, spalla, agnelo, malloreddus...",6,4
7,"Osteria la Spiga - Seattle, WA","[ragu, piadina, cestino, stuzzichini, ida, gno...",7,4
4,"I Ricchi - Washington, DC","[romagnola, salsiccie, trota, buratta, paillar...",14,3
8,"Via dei Tribunali - Seattle, WA","[cappuccino, caffe, espresso, puttanesca, dant...",11,3
9,"Il Terrazzo Carmine - Seattle, WA","[affogati, ortolano, spaghettini, asparaci, an...",15,3
3,"Otello - Washington, DC","[cappuccino, episelli, trota, espresso, mirell...",18,2
6,"Valentinos - Nashville, TN","[arugula, parmesan, manhattan, scottish, espre...",18,2
0,"Ristorante Piccolo - Washington, DC","[arugula, cioppino, altopiano, parigina, scarp...",26,1
2,"Portofino - Arlington, VA","[vinci, omaggi, veronese, picatta, cioppino, a...",24,1


In [166]:
for key, value in sorted(dic_scoring.items(), key=lambda x:x[1]):
    print("------------------")
    print("Restaurant: ", key)
    print("# Errors: ", value[0][0])
    print("Errors: ", value[0][1])
    print('Score: ', value[1])

------------------
Restaurant:  Il Canale - Washington, DC
# Errors:  5
Errors:  ['georgetown', 'rolle', 'tunnarella', 'ciliegina', 'imbottito']
Score:  5
------------------
Restaurant:  La Storia - Chicago, IL
# Errors:  6
Errors:  ['neri', 'orecchitte', 'spalla', 'agnelo', 'malloreddus', 'sautee']
Score:  4
------------------
Restaurant:  Osteria la Spiga - Seattle, WA
# Errors:  7
Errors:  ['ragu', 'piadina', 'cestino', 'stuzzichini', 'ida', 'gnocco', 'stridoli']
Score:  4
------------------
Restaurant:  Via dei Tribunali - Seattle, WA
# Errors:  11
Errors:  ['cappuccino', 'caffe', 'espresso', 'puttanesca', 'dante', 'tribunali', 'campania', 'specialita', 'lupino', 'americano', 'spaccanapoli']
Score:  3
------------------
Restaurant:  I Ricchi - Washington, DC
# Errors:  14
Errors:  ['romagnola', 'salsiccie', 'trota', 'buratta', 'paillar', 'costoletta', 'tortelloni', 'cavatelli', 'tagliarini', 'neri', 'bietola', 'allaglione', 'strascicate', 'vegetariano']
Score:  3
------------------